In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gudhi as gd
import sys
sys.path.append('../topotests/')
from scipy.stats import multivariate_normal

In [2]:
from distributions import *

In [3]:
rv = MultivariateDistributionJitter([st.poisson(1)], label="Poisson(1)")

In [14]:
rv = MultivariateDistribution([st.poisson(1)], label="Poisson(1)")
X = rv.rvs(100)
alpha = gd.AlphaComplex(points = X)
alpha_tree = alpha.create_simplex_tree()

In [49]:
rips = gd.RipsComplex(points = X)
rips_tree = rips.create_simplex_tree()
#rips_filtr = rips_tree.get_filtration()

In [53]:
ecc = {}
for s, f in rips_tree.get_filtration():
    dim = len(s)-1
    ecc[f] = ecc.get(f, 0) + (-1)**dim

In [7]:
def compute_ECC(point_cloud, complex_type='alpha'):

    if complex_type == 'alpha':
        comp = gd.AlphaComplex(points=point_cloud)
    else:
        comp = gd.RipsComplex(points=point_cloud)

    simplex_tree = comp.create_simplex_tree()

    ecc = {}

    for s, f in simplex_tree.get_filtration():
        dim = len(s) - 1
        ecc[f] = ecc.get(f, 0) + (-1) ** dim

    # remove the contributions that are 0
    to_del = []
    for key in ecc:
        if ecc[key] == 0:
            to_del.append(key)
    for key in to_del:
        del ecc[key]

    ecc_sort = np.array(sorted(list(ecc.items()), key=lambda x: x[0]))
    ecc_sort[:, 1] = np.cumsum(ecc_sort[:, 1])
    return ecc_sort

In [28]:
y = compute_ECC(X, complex_type='rips')